In [ ]:
Pkg.clone("https://github.com/mpf/Toms566.jl.git")

In [2]:
using Toms566

p=Problem(2)

x0=p.x0 # starting point
grad_x0=p.grd(p.x0) #gradient at point [x,x_2,x_3,...,x_n]
hes_x0=p.hes(p.x0) #grad^2 (heshian) at point [x,x_2,x_3,...,x_n]
obj_x_0=p.obj(p.x0) #objective value at point [x,x_2,x_3,...,x_n]

0.7790700756559702

In [49]:
hes_x0
svd(hes_x0)

(
6x6 Array{Float64,2}:
 -0.23067    0.169093   0.180489    0.540161    0.674745   0.372247
 -0.236695   0.259715   0.495405   -0.457177   -0.244582   0.601885
  0.366212  -0.798933   0.38575    -0.0900557   0.194071   0.18171 
 -0.47132   -0.451469  -0.536131    0.135403   -0.260077   0.447909
 -0.465321  -0.181795   0.534032    0.418187   -0.380833  -0.381212
  0.563905   0.16967    0.0238239   0.545794   -0.484315   0.346703,

[19.694655718347803,10.864394660741004,8.500168766773363,5.427530236008022,1.524208336049016,1.5189583620351783],
6x6 Array{Float64,2}:
 -0.23067    0.169093  -0.180489    0.540161    0.674745  -0.372247
 -0.236695   0.259715  -0.495405   -0.457177   -0.244582  -0.601885
  0.366212  -0.798933  -0.38575    -0.0900557   0.194071  -0.18171 
 -0.47132   -0.451469   0.536131    0.135403   -0.260077  -0.447909
 -0.465321  -0.181795  -0.534032    0.418187   -0.380833   0.381212
  0.563905   0.16967   -0.0238239   0.545794   -0.484315  -0.346703)

In [48]:
(U, Σ, V)=svd(hes_x0);

for k=1:length(Σ)
    if inv(Σ[k])<0
        Σ[k]=0;
    end
end

Σ = diagm(Σ)
U*Σ*V'


6x6 Array{Float64,2}:
  3.14874  -1.13981   -3.89039   2.01055   2.01055  -1.38066
 -1.13981   0.425374  -5.60088   2.53237  -1.13981  -3.74102
 -3.89039  -5.60088    8.36237   2.01055  -3.74102   2.01055
  2.01055   2.53237    2.01055   4.04406   8.36237  -5.60088
  2.01055  -1.13981   -3.74102   8.36237   3.14874  -3.89039
 -1.38066  -3.74102    2.01055  -5.60088  -3.89039   8.36237

In [33]:
hes_x0

3x3 Array{Float64,2}:
   200.0   -1591.55     0.0 
 -1591.55    506.606  318.31
     0.0     318.31   202.0 

In [12]:
function newtmin(obj, x0; max_iter=2500, tol=1e-6)
    x=[]
    g=[]
    h=[]
    iter_stop=0
    alpha=1
    
        x=push!(x,x0);

    for iter=1:max_iter
        
        neg_eig_test=0;
        
        g1 = obj.grd(x[end])
        h1 = obj.hes(x[end])
        
        #make sure H is positive definate
        #do this by taking the SVD, invert the sigma, then make all the values positve through some means, 
        #then set H_bar = U*sigma_inverse* V
        
        (U, Σ, V)=svd(h1);
        
        for k=1:length(Σ)
            if Σ[k]<0
                Σ[k]=0;
                neg_eig_test=1;
            end
        end
        
        if neg_eig_test==1
            Σ = diagm(Σ);
            h1=U*Σ*V';
            print("Negative eigenvalue encoutered in H")
        end
        
        
        g=push!(g,g1);
        h=push!(h,h1);
        

        
        x=push!(x, x[end] + alpha*( inv(h1)*(-g1) )) #x(k+1)=x(k)+alpha*d(k), where d(k)= solution of (hd=-g)
        
        if norm(g[end])<tol || iter>2 && abs(norm(g[end])-norm(g[end-1])) < 1e-15
            iter_stop=iter;
            break
        end
    end

    return (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)

end

newtmin (generic function with 1 method)

In [27]:
p=Problem(18)

newtmin(p,p.x0)

 ** On entry to DLASCLS parameter number  4 had an illegal value
 ** On entry to DLASCLS parameter number  4 had an illegal value


LoadError: LoadError: Base.LinAlg.LAPACKException(23)
while loading In[27], in expression starting on line 3

In [ ]:
# Problem #   x[end]
# norm(g[end]),obj.obj(x[end]),iter_stop
#
#Note if iter_stop is 0 then a solution was not found. . . 
#------------------------------------------------------------------------------------------------------------------------------
# Problem 9 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
# ************ERROR***************

# Problem 10 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([1.0e6,2.0e-6],
#1.7763568394019445e-9,0.0,740)

# Problem 11 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
#************ERROR***************

# Problem 12 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([-0.8203029592211489,11.180372142246291,-6.583308445312273],
#8.528600282736342e-7,32.835000095286894,674)

# Problem 13 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([-8.26522e-17,-2.25496e-15,-1.02607e-15,3.31861e-15,3.08293e-15,-9.44966e-16,2.54012e-15,-1.36046e-15,1.81935e-15,-1.50898e-16  …  -4.60169e-16,5.93169e-15,-5.30974e-15,1.76727e-15,4.74812e-15,-4.43165e-16,-5.37691e-15,-9.70829e-16,5.72889e-15,5.47033e-16],
#2.9172960754798295e-9,0.0,13)

# Problem 14 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
#214.1,-181.756,31214.1,-181.756,31214.1,1.0,1.0],
#6.536145832960807e19,1.5147975697323924e24,0)

# Problem 15 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************I think the exited due to gradient not changing by measurable amount**************
#([-4.28776e33,9.42258e29,4.71129e29,-4.2872e33,-4.64417e33,6.41602e26,3.20801e26,-4.64383e33,-1.29277e27,7.59459e19  …  3.07952e8,-1.3898e27,-2.68654e29,2.65404e22,1.32702e22,-2.68654e29,6.28473e27,6.53901e15,1.08432e14,6.28473e27],
#9.898366845755256e101,4.542742026847571e134,2215)

# Problem 16 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([0.0,1.0],
#0.0,14.203125,2)

# Problem 17 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([-0.9679740249375894,0.9471391408178346,-0.969516310331595,0.9512476657923326],
#2.209969838187114e-7,7.876967165176868,14)

# Problem 18 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
#************ERROR***************

p=Problem(3)

newtmin(p, p.x0)


In [ ]:
# Problem #   x[end]
# norm(g[end]),obj.obj(x[end]),iter_stop
#
#Note if iter_stop is 0 then a solution was not found. . . 
#------------------------------------------------------------------------------------------------------------------------------

# Problem 1 
#([1.0,9.623971777241715e-23,1.5541301993193564e-22],
#1.5031454551348873e-10,2.465620036363938e-44,34)

# Problem 2 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
# ([110.58764417028193,-0.8911596223816917,-1.0451210209117733e6,-0.19746348605916325,33.426111922868564,489.55665776016485],
#8.06573891610286,3.4729647629773392,0)

# Problem 3 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([0.3989561378387567,1.0000190844878059,5.732690006101047e-20],
#9.701953934166454e-11,1.1279327696190214e-8,3)

# Problem 4 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([-0.00010003002051701595,-0.00010003002051701596],
#2.087691668169917e-14,2.000000040020681,17)

# Problem 5 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
# ([0.9999999999713464,10.00000000022855,1.0000000000180065],
#4.4387809906411776e-7,5.0566242968258996e-23,9)

# Problem 6 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
#([-875.363,50.9299,2.05697,-31.7899,91.1362,181.32,-10.4097,-233.889,-230.617,164.751  …  23.7955,30.728,-45.0972,-34.3397,-26.2373,-12.0429,65.18,-536.708,-40.4993,564.106];
#19842.370946806597,1.657205138963716e6,0)

# Problem 7 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop) **************DID NOT FIND SOLUTION**************
#([5.75736265923876e27,-2.0979027425589095e29,1.189759559553719e30,-2.182792565627064e30,8.352311691447432e29,1.0207851679422616e30,-4.998438851141209e29,4.415239152935261e28,-2.0835032082229147e29],
#1.2430396861371629e84,8.199302649207593e111,0)

# Problem 8 (x[end],norm(g[end]),obj.obj(x[end]),iter_stop)
#([0.0645592,0.0645591,0.0645591,0.0645591,0.0645591,0.064559,0.064559,0.0645591,0.064559,0.0645591  …  0.0645591,0.0645591,0.0645591,0.0645591,0.0645591,0.0645591,0.0645591,0.0645591,0.0645591,0.0645591],
#1.6181364996315994e-8,0.0005250350834856768,20)

In [14]:
for i = 1:10
    x = [x; i]
end

In [15]:
x'

1x10 Array{Any,2}:
 1  2  3  4  5  6  7  8  9  10

In [2]:
100

100